In [1]:
cd ..

/home/aiteam/tykim/generative/Generative_Model_Labs/src


In [2]:
import torch
import torch.nn as nn
from models.stylegan2 import SynthesisBlock
import utils.style_misc as misc

/home/aiteam/miniconda3/envs/3dfm/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
batch_size = 16
in_channels=32
out_channels=64
w_dim=512
resolution=128
img_channels=3


img = torch.randn((batch_size, img_channels, resolution//2, resolution//2)).to('cuda')
ws = torch.randn((batch_size, 3, w_dim)).to('cuda')
x = torch.randn((batch_size, in_channels, resolution//2, resolution//2)).to('cuda')

test_blk = SynthesisBlock(in_channels=32, out_channels=64, w_dim=512, resolution=128, img_channels=3, is_last=True, use_fp16=True, fp16_channels_last=True).to('cuda')

In [4]:
test_blk.num_conv, test_blk.num_torgb

(2, 1)

In [4]:
test_blk.resample_filter.dtype

torch.float32

In [14]:
out_x, out_img = test_blk(x, img, ws)

In [15]:
out_x.shape

torch.Size([16, 64, 128, 128])

In [16]:
out_img.shape

torch.Size([16, 3, 128, 128])

In [5]:
from types import MethodType

In [6]:
def forward2(self, x, img, ws, force_fp32=False, fused_modconv=None, update_emas=False, **layer_kwargs):
    _ = update_emas # unused
    misc.assert_shape(ws, [None, self.num_conv + self.num_torgb, self.w_dim])
    w_iter = iter(ws.unbind(dim=1))
    dtype = torch.float16 if self.use_fp16 and not force_fp32 else torch.float32
    memory_format = torch.channels_last if self.channels_last and not force_fp32 else torch.contiguous_format
    if fused_modconv is None:
        with misc.suppress_tracer_warnings():  # this value will be treated as a constant
            fused_modconv = (not self.training) and (dtype == torch.float32 or int(x.shape[0]) == 1)

    # Input.
    if self.in_channels == 0:
        x = self.const.to(dtype=dtype, memory_format=memory_format)
        x = x.unsqueeze(0).repeat([ws.shape[0], 1, 1, 1])
    else:
        misc.assert_shape(x, [None, self.in_channels, self.resolution // 2, self.resolution // 2])
        x = x.to(dtype=dtype, memory_format=memory_format)

    # Main layers.
    if self.in_channels == 0:
        x = self.conv1(x, next(w_iter), fused_modconv=fused_modconv, **layer_kwargs)
    elif self.architecture == "resnet":
        y = self.skip(x, gain=np.sqrt(0.5))
        x = self.conv0(x, next(w_iter), fused_modconv=fused_modconv, **layer_kwargs)
        x = self.conv1(x, next(w_iter), fused_modconv=fused_modconv, gain=np.sqrt(0.5), **layer_kwargs)
        x = y.add_(x)
    else:
        x = self.conv0(x, next(w_iter), fused_modconv=fused_modconv, **layer_kwargs)
        x = self.conv1(x, next(w_iter), fused_modconv=fused_modconv, **layer_kwargs)

    return x

In [7]:
test_blk.forward2 = MethodType(forward2, test_blk)

In [8]:
x.device

device(type='cuda', index=0)

In [9]:
x_out = test_blk.forward2(x, img, ws)

Setting up PyTorch plugin "upfirdn2d_plugin"... Done.
Setting up PyTorch plugin "bias_act_plugin"... Done.


In [10]:
x_out.dtype

torch.float16

In [21]:
test_x = torch.randn(3, 10, 32, 32).to('cuda', dtype=torch.float16, memory_format=torch.channels_last)
test = nn.Conv2d(10, 20, 3).to('cuda', dtype=torch.float16, memory_format=torch.channels_last)

test(test_x).dtype

torch.float16

In [25]:
test_x = torch.randn(3, 10, 32, 32).cuda()
test = nn.Conv2d(10, 20, 3).cuda()

with torch.autocast(device_type='cuda', dtype=torch.float16):
    output = test(test_x)
    print(output.dtype)

torch.float16


In [23]:
output.dtype

torch.float32